In [9]:
#import root_finder as rf
from ad import DiffObj, Variable, VectorFunction
from ad import MathOps as mo
import random
from scipy.interpolate import interp1d
random.seed(13)

import numpy as np
import matplotlib.pyplot as plt
import math
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
x1 = Variable('x1',1)
x2 = Variable('x2',4)
x3 = Variable('x3',2)
f1 = x1**2 - 2*x1+x2**2-x3+1
f2 = x1*x2**2-x1-3*x2+x2*x3+2
f3 = x1*x3**2-3*x3+x2*x3**2+x1*x2
f4 = x1**2 - 2*x1+x2**2-x3+1


v1 = VectorFunction([f1,f2])
v2 = VectorFunction([f2,f3])
v3 = VectorFunction([f1,f2])

v4 = VectorFunction([x1,x1])
v5 = VectorFunction([x1,x2])

#print(f1==f2)
print(v4 == v5)

#x3.get_dict_val()

False


In [ ]:
def pretty_print_results(results_list):
    print("RootFinder found a total of {} roots for the function\n".format(len(results_list))) 
    for root in results_list:   
        print("Found root: {}".format(root[0]))
        print("Function value at root: {}".format(root[1]))
        print("Number of iterations needed to find the root: {}".format(root[2]))
        print("List of error values:{}".format(root[3]))
        
        plt.title("Error values vs. iteratation number")
        plt.xlabel("Iteration Number")
        plt.ylabel("Error Value")
        print()
        
        # add smooth line
        x = list(range(len(root[3])))
        y = root[3]
        if(len(x)>2):
            x_new = np.linspace(min(x), max(x),500)
            f = interp1d(x, y, kind='quadratic')
            y_smooth=f(x_new)
            plt.plot (x_new,y_smooth)      
        plt.scatter(x,y)
    
    labels = ["Error for root {}".format(i+1) for i in range(len(results_list))]
    plt.legend(labels)
    plt.show()

In [ ]:
# Very basic test 
x = Variable('x')
y = Variable('y')

test_f0 =  x+1

results_t0= rf.vectorNewton(test_f0, num_starting_vals=10, verbose=True)
results_t0
pretty_print_results(results_t0)

roots = [result[0]['x'] for result in results_t0]
val_at_root = [result[1] for result in results_t0]

plt.title("Function and roots from rootfinder")
x_plt = np.arange(min(roots)-5, max(roots)+5,1)
y_plt= x_plt+1
x_new = np.linspace(min(x_plt), max(x_plt),500)
f = interp1d(x_plt, y_plt)
y_smooth=f(x_new)
plt.plot (x_new,y_smooth) 
plt.axhline(y=0,color='r')
plt.plot(roots, val_at_root, marker='o', markersize=10, color="black")
plt.legend(["Function","Line at zero","root"])
#plt.scatter(x_plt,y_plt)
plt.show()

for i, root in enumerate(roots):
    r = test_f0.get_val({'x':root})
    print("Evaluation of our function at root number {}: {}".format(i,r))

In [ ]:
x = Variable('x')
test_f1 = x**3+1
results_t1= rf.vectorNewton(test_f1, num_starting_vals=10, verbose=True)
pretty_print_results(results_t1)

roots = [result[0]['x'] for result in results_t1]
val_at_root = [result[1] for result in results_t1]

plt.title("Function and roots from rootfinder")
x_plt = np.arange(min(roots)-5, max(roots)+5,1)
y_plt= x_plt**3 +1
x_new = np.linspace(min(x_plt), max(x_plt),500)
f = interp1d(x_plt, y_plt)
y_smooth=f(x_new)
plt.plot (x_new,y_smooth) 
plt.axhline(y=0,color='r')
plt.plot(roots, val_at_root, marker='o', markersize=10, color="black")
plt.legend(["Function","Line at zero","root"])
plt.show()

for i, root in enumerate(roots):
    r = test_f1.get_val({'x':root})
    print("Evaluation of our function at root number {}: {}".format(i,r))

In [ ]:
# Test a function with many roots 
x = Variable('x')
test_f2 = mo.sin(x)+0.5
results_t2= rf.vectorNewton(test_f2, num_starting_vals=20, starting_val_range = (-10,10), verbose=True)
pretty_print_results(results_t2)

roots = [result[0]['x'] for result in results_t2]
val_at_root = [result[1] for result in results_t2]

plt.title("Function and roots from rootfinder")
x_plt = np.arange(min(roots)-5, max(roots)+5,1)
y_plt= np.sin(x_plt)+0.5
x_new = np.linspace(min(x_plt), max(x_plt),500)
f = interp1d(x_plt, y_plt, kind='quadratic')
y_smooth=f(x_new)
plt.plot (x_new,y_smooth) 
plt.axhline(y=0,color='r')
plt.scatter(roots, val_at_root, marker='o', s=30, color="black")
plt.legend(["Function","Line at zero","roots"])
plt.show()

for i, root in enumerate(roots):
    r = test_f2.get_val({'x':root})
    print("Evaluation of our function at root number {}: {}".format(i,r))

In [ ]:
# Test a vector valued function
x = Variable('x')
y = Variable('y')

test_f3 = VectorFunction([x**2+y, y+x])
results_t3= rf.vectorNewton(test_f3, num_starting_vals=10, verbose=True)
pretty_print_results(results_t3)

roots = [result[0] for result in results_t3]
val_at_root = [result[1] for result in results_t3]

for i, root in enumerate(roots):
    r = test_f3.get_val(root)
    print("Evaluation of our function at root number {}: {}".format(i,r))

In [ ]:
# Test a second vector valued function
x1 = Variable('x1')
x2 = Variable('x2')
x3 = Variable('x3')

f1 = x1**2 - 2*x1+x2**2-x3+1
f2 = x1*x2**2-x1-3*x2+x2*x3+2
f3 = x1*x3**2-3*x3+x2*x3**2+x1*x2

test_f4 = VectorFunction([f1,f2,f3])
results_t4= rf.vectorNewton(test_f4, num_starting_vals=5, verbose=True)
pretty_print_results(results_t4)

roots = [result[0] for result in results_t4]
val_at_root = [result[1] for result in results_t4]

for i, root in enumerate(roots):
    r = test_f4.get_val(root)
    print("Evaluation of our function at root number {}: {}".format(i,r))